In [35]:
import matplotlib.pyplot as plt
import numpy as np
from keras import layers, models
from keras.datasets import imdb, mnist
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import seaborn as sns

%matplotlib inline

In [2]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)

In [6]:
X_train.shape, X_test.shape

((25000,), (25000,))

In [7]:
# Define a fn that applies one-hot encoding
def vectorize_sequences(sequences, dimension=10000):
    """
    Applies one-hot encoding on sequences.
    """
    resuls = np.zeros((len(sequences), dimension))
    
    for i, sequence in enumerate(sequences):
        resuls[i, sequence] = 1
        
    return resuls

# Convert sequences into one-hot encoding vectors
X_train = vectorize_sequences(X_train)
X_test = vectorize_sequences(X_test)
X_train.shape, X_test.shape

((25000, 10000), (25000, 10000))

In [12]:
# Building fully connected NN using Sequential model
model = models.Sequential()
model.add(layers.Dense(16, activation="relu", input_shape=(10000,)))
model.add(layers.Dense(16, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

# Compile the model
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["acc"])

# Fit the model
history = model.fit(X_train,
                    y_train,
                    batch_size=256,
                    epochs=1)

Epoch 1/1
25000/25000 [==============================] - 2s 70us/step - loss: 0.3901 - acc: 0.8413


In [13]:
# Building fully connected NN using functional API
input_tensor = layers.Input(shape=(10000,))
x = layers.Dense(16, activation="relu")(input_tensor)
x = layers.Dense(16, activation="relu")(x)
output_tensor = layers.Dense(1, activation="sigmoid")(x)
model = models.Model(inputs=input_tensor, outputs=output_tensor)

# Compile the model
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["acc"])

# Fit the model
history = model.fit(X_train,
                    y_train,
                    batch_size=256,
                    epochs=1)

Epoch 1/1
25000/25000 [==============================] - 2s 72us/step - loss: 0.3865 - acc: 0.8514


In [15]:
# Create a multi-input model
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_size = 500

# Create text related input layers
text_input = layers.Input(shape=(10000,), dtype="int32", name="text")
embedded_text = layers.Embedding(64, text_vocabulary_size)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

# Create question related layers
question_input = layers.Input(shape=(10000,), dtype="int32", name="question")
embedded_question = layers.Embedding(64, question_vocabulary_size)(question_input)
encoded_question = layers.LSTM(32)(question_text)

# Concatenate both inputs
concatenated = layers.concatenate([encoded_text, encoded_question])

# Build dense layer
answer = layers.Dense(answer_size, activation="softmax")(concatenated)

# Compile the model
answer.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["acc"])

# Fit the model either using dictionary or list
history = model.fit([text, question],
                    answers,
                    batch_size=256,
                    epochs=1)

history = model.fit({"text": text, "question": question},
                    answers,
                    batch_size=256,
                    epochs=1)

In [ ]:
# Create multi-output model using social media posts
vocabulary_size = 50000
num_income_groups = 10

posts_input = layers.Input(shape=(None,), dtype="int32", name="posts")
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_posts)
x = layers.MaxPool1D(5)(x)
x = layers.Conv1D(256, 5, activation="relu")(x)
x = layers.Conv1D(256, 5, activation="relu")(x)
x = layers.MaxPool1D(5)(x)
x = layers.Conv1D(256, 5, activation="relu")(x)
x = layers.Conv1D(256, 5, activation="relu")(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(128, activation="relu")(x)

age_predictions = layers.Dense(1, name="age")(x)
income_predictions = layers.Dense(num_income_groups, activation="softmax", name="income")(x)
gender_predictions = layers.Dense(1, activation="sigmoid", name="gender")(x)

model = models.Model(posts_input, [age_predictions, income_predictions, gender_predictions])

model.compile(optimizer="rmsprop",
              loss=["mse", "categorical_crossentropy", "binary_crossentropy"],
              loss_weights=[0.25, 1, 10])

model.compile(optimizer="rmsprop",
              loss={"age": "mse", "income": "categorical_crossentropy", "gender": "binary_crossentropy"},
              loss_weights={"age": 0.25, "income": 1, "gender": 10})

model.fit(posts,
          [age_targets, income_group_targets, gender_targets],
          epochs=10, batch_size=32)
model.fit(posts,
          {"age": age_targets, "income": income_group_targets, "gender": gender_targets},
          epochs=10, batch_size=32)

In [16]:
from keras.applications import InceptionV3

In [ ]:
InceptionV3()

In [17]:
import keras

In [ ]:
keras.callbacks.ModelCheckpoint()

In [19]:
# Recreate model on imdb dataset to visualize whats happening during training
max_features = 2000
max_len = 500
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_train.shape, X_test.shape

((25000, 500), (25000, 500))

In [20]:
model = models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len, name="embed"))
model.add(layers.Conv1D(32, 7, activation="relu"))
model.add(layers.MaxPool1D(5))
model.add(layers.Conv1D(32, 7, activation="relu"))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(1, activation="sigmoid"))

model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["acc"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
_________________________________________________________________


In [21]:
%mkdir my_log_dir

In [30]:
callbacks_list = [
    keras.callbacks.TensorBoard(
        log_dir="my_log_dir", histogram_freq=1, embeddings_freq=1
    ), keras.callbacks.EarlyStopping(
        patience=1)
]

history =  model.fit(X_train, y_train,
                     batch_size=128,
                     epochs=20,
                     validation_split=0.2,
                     callbacks=callbacks_list)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 63s 3ms/step - loss: 0.0103 - acc: 0.9985 - val_loss: 0.6140 - val_acc: 0.8592
Epoch 2/20
20000/20000 [==============================] - 70s 4ms/step - loss: 0.0059 - acc: 0.9992 - val_loss: 0.6848 - val_acc: 0.8590


In [32]:
# Load the data
(train_data, train_labels), (test_data, test_labels) = mnist.load_data()
train_data.shape, train_labels.shape, test_data.shape, test_labels.shape

11493376/11490434 [==============================] - 3s 0us/step


((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [54]:
X_train = train_data.reshape((-1, 28, 28, 1)) / 255
X_test = test_data.reshape((-1, 28, 28, 1)) / 255
X_train.shape, X_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [55]:
y_train = to_categorical(train_labels)
y_test = to_categorical(test_labels)
y_train.shape, y_test.shape

((60000, 10), (10000, 10))

In [56]:
# Build Conv network
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1)))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(10, activation="softmax"))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_8 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 64)                36928     
__________

In [57]:
# Compile it
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["acc"])

# Fit the model
history = model.fit(X_train,
                    y_train,
                    epochs=5,
                    batch_size=64)

test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy is: {:.2f}%".format(test_acc * 100))

Epoch 1/5
60000/60000 [==============================] - 29s 487us/step - loss: 0.1714 - acc: 0.9459
Epoch 2/5
60000/60000 [==============================] - 28s 468us/step - loss: 0.0477 - acc: 0.9852
Epoch 3/5
60000/60000 [==============================] - 28s 466us/step - loss: 0.0336 - acc: 0.9893
Epoch 4/5
60000/60000 [==============================] - 29s 478us/step - loss: 0.0243 - acc: 0.9927
Epoch 5/5
10000/10000 [==============================] - 2s 180us/step
Test accuracy is: 99.18%


In [58]:
# Build Conv network
model = models.Sequential()
model.add(layers.SeparableConv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1)))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.SeparableConv2D(64, (3, 3), activation="relu"))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.SeparableConv2D(64, (3, 3), activation="relu"))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(10, activation="softmax"))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_7 (Separabl (None, 26, 26, 32)        73        
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
separable_conv2d_8 (Separabl (None, 11, 11, 64)        2400      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
separable_conv2d_9 (Separabl (None, 3, 3, 64)          4736      
_________________________________________________________________
flatten_9 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 64)                36928     
__________

In [59]:
# Compile it
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["acc"])

# Fit the model
history = model.fit(X_train,
                    y_train,
                    epochs=5,
                    batch_size=64)

test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy is: {:.2f}%".format(test_acc * 100))

Epoch 1/5
60000/60000 [==============================] - 21s 347us/step - loss: 0.4237 - acc: 0.8693
Epoch 2/5
60000/60000 [==============================] - 20s 338us/step - loss: 0.1092 - acc: 0.9664
Epoch 3/5
60000/60000 [==============================] - 21s 344us/step - loss: 0.0723 - acc: 0.97722s - ETA: 0s - loss: 0.0721 - 
Epoch 4/5
60000/60000 [==============================] - 21s 342us/step - loss: 0.0573 - acc: 0.98200s - loss: 0.0572 - acc: 0.98
Epoch 5/5
10000/10000 [==============================] - 2s 152us/step
Test accuracy is: 98.11%


In [61]:
keras.utils.plot_model(model)

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.